In [30]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from transformers import *

p = '/Users/numlaut/dsi/projects/7.0 project 4/West_Nile_Kaggle/assets/input/'
spraydf =pd.read_csv(p+'spray.csv')
traindf =pd.read_csv(p+'train.csv')
ktestdf = pd.read_csv(p+'test.csv')
weatherdf = pd.read_csv(p+'clean_weather.csv')

In [31]:
import time
def date_to_utc(x):
    return int(time.mktime(time.strptime(x,'%Y-%m-%d')))

def utc_to_date(x):
    return time.strftime('%Y-%m-%d',time.gmtime(x))

In [32]:
print(traindf.columns)

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent'],
      dtype='object')


In [33]:
# date_to_utc defined above
utc = ColumnMapper(func=date_to_utc,column='date',name='utc',drop=False) # keep 'date' column and utc

In [34]:
## building weatherdf ##
#selection station 1 data
mask = (weatherdf['Station']==1)

# compile with renaming
weather = weatherdf[mask].rename(columns={'Date':'date'})

#weather processing
pipe = []
pipe.append(('utc',utc))

#transform w/pipe
prep_weather = Pipeline(pipe)
weather = prep_weather.fit_transform(weather)

In [35]:
## building train df ##

# define renamed columns
rn = {}
rn['Date']='date'
rn['Latitude']='lat'
rn['Longitude']='long'
rn['AddressNumberAndStreet']='address'
rn['Species']='species'

# define selected columns
feats = 'date address species lat long'.split()

# define y
target = 'WnvPresent'.split()

# compile with renaming
train = traindf.rename(columns=rn)[ feats + target ] #y included
ktest = ktestdf.rename(columns=rn)[feats]

In [36]:
## define transformations ##

# loc
from collections import namedtuple
coord = namedtuple('coordinate',['lat','long'])
loc = ColumnApplier(lambda row: coord(row['lat'],row['long']), name='loc', axis=1)

# date_to_utc defined above
utc = ColumnMapper(func=date_to_utc,column='date',name='utc',drop=False) # keep 'date' column and utc

#merge w/weather from weatherdf by DATE
merge_weather = DfMerger(weather,on=['utc','date'],how='left',copy=True,validate='m:1')

#dummy trap locations
DummyEncoder

transformers.DummyEncoder

In [37]:
#feature prep
pipe = []
pipe.append(('loc',loc))
pipe.append(('utc',utc))
pipe.append(('merge_weather',merge_weather))
prep_features = Pipeline(pipe)

In [38]:
#build sets
train = prep_features.fit_transform(train) #for training
ktest = prep_features.fit_transform(test) #for kaggle submission

#define X and y
y = train[target]
X = train.drop(target,axis=1)

feats = list(train.columns)#recatalogue cols for later use if necessary

#train test split
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,stratify=y)

NameError: name 'weather' is not defined

In [10]:
def submit(yhat,name='WnvPresent'):
    submit = pd.DataFrame(yhat,columns=['WnvPresent'])
    submit.index +=1
    submit.index.name = 'Id'
    submit.to_csv(str('./results_'+name+'.csv'))
    return submit['WnvPresent'].value_counts()